# Importowanie bibliotek

In [3]:
# import keras
# import numpy as np
# import tensorflow as tf
# import matplotlib as plt
# import sklearn

import pandas as pd
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, Flatten, Conv2D, MaxPool2D, Dropout
from keras import layers
from keras.optimizers import SGD, Adam
import numpy as np
import matplotlib.pyplot as plt
from imblearn.under_sampling import NearMiss

# Wczytujemy zbiór danych


In [4]:
data = pd.read_csv("A_Z Handwritten Data.csv").astype('float32')
data.head()

,0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,...,0.639,0.640,0.641,0.642,0.643,0.644,0.645,0.646,0.647,0.648
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# Odzielanie etykiet od obrazu

In [5]:
X = data.drop('0',axis = 1)
y = data['0']
X.head()
y.head()

0    0.0
1    0.0
2    0.0
3    0.0
4    0.0
Name: 0, dtype: float32

# Reshapowanie danych testowych i treningowych do standardów CNN

In [6]:
nM = NearMiss()
X, y = nM.fit_resample(X, y)

X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size = 0.2)

print("Wymiary treningowe przed reshapem: ", X_train.shape)
print("Wymiary testowe przed reshapem: ", X_test.shape)

X_train = np.reshape(X_train.values, (X_train.shape[0], 28, 28, 1))
X_test = np.reshape(X_test.values, (X_test.shape[0], 28, 28, 1))

print("Wymiary treningowe po reshapem: ", X_train.shape)
print("Wymiary testowe po reshapie: ", X_test.shape)


Wymiary treningowe przed reshapem:  (23296, 784)
Wymiary testowe przed reshapem:  (5824, 784)
Wymiary treningowe po reshapem:  (23296, 28, 28, 1)
Wymiary testowe po reshapie:  (5824, 28, 28, 1)


# Stworzenie słownika

In [7]:
word_dict = {0:'A',1:'B',2:'C',3:'D',4:'E',5:'F',6:'G',7:'H',8:'I',9:'J',10:'K',11:'L',12:'M',13:'N',14:'O',15:'P',16:'Q',17:'R',18:'S',19:'T',20:'U',21:'V',22:'W',23:'X', 24:'Y',25:'Z'}

# Zamiana etykiety z pojedńczej liczby na wektor.

In [8]:
train_yOHE = to_categorical(Y_train, num_classes = 26, dtype='int')
print("Nowy rozmiar etykiety treningowej: ", train_yOHE.shape)

test_yOHE = to_categorical(Y_test, num_classes = 26, dtype='int')
print("Nowy rozmiar etykiety treningowej: ", test_yOHE.shape)

Nowy rozmiar etykiety treningowej:  (23296, 26)
Nowy rozmiar etykiety treningowej:  (5824, 26)


# Tworzenie CNN

In [9]:
model = Sequential()
#CNN
# input -> conv -> maxpool -> conv -> maxpool ......->flattened vector-> 
#.                        hidden layer -> hidden layer -> softmax layer

#Wyciągamy cechy z obrazu, a następnie zmniejszamy jego rozmiar czterokrotnie. Filtry zwiększają się dwukrotnie, bo zapisujemy
#z każdym krokiem więcej cech.
model.add(Conv2D(filters=32, kernel_size=(3, 3), activation='relu', input_shape=(28,28,1)))
model.add(MaxPool2D(pool_size=(2, 2), strides=2))

model.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu', padding = 'same'))
# model.add(MaxPooling2D(pool_size=(2, 2), strides=2))

model.add(Conv2D(filters=128, kernel_size=(3, 3), activation='relu', padding = 'same'))
# model.add(MaxPooling2D(pool_size=(2, 2), strides=2))
model.add(Dropout(0.2))
model.add(Conv2D(filters=256, kernel_size=(3, 3), activation='relu', padding = 'same'))
model.add(MaxPool2D(pool_size=(2, 2), strides=2))

model.add(Flatten())

model.add(Dense(128,activation ="relu"))
model.add(Dropout(0.2))
model.add(Dense(256,activation ="relu"))

model.add(Dense(26,activation ="softmax"))

imageAug = Sequential([
    layers.RandomZoom(height_factor=(-0.3, 0.3), width_factor=(-0.3, 0.3)),
    layers.RandomRotation(0.4)])

for i in range(len(X_train)):
  X_train[i] = imageAug(X_train[i])


# Kompilacja i trening CNN

In [ ]:
model.compile(optimizer = Adam(learning_rate=0.01), loss='categorical_crossentropy', metrics=['accuracy'])

history = model.fit(X_train, train_yOHE, epochs=10,  batch_size=128, validation_data = (X_test,test_yOHE))

Epoch 1/10
 47/182 [======>.......................] - ETA: 2:57 - loss: 2.5298 - accuracy: 0.3672

# Zapis model i jego spis

In [ ]:
model.summary()
model.save(r'model_hand2.h5')

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_3 (Conv2D)           (None, 26, 26, 32)        320       
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 13, 13, 32)       0         
 2D)                                                             
                                                                 
 conv2d_4 (Conv2D)           (None, 13, 13, 64)        18496     
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 6, 6, 64)         0         
 2D)                                                             
                                                                 
 flatten_1 (Flatten)         (None, 2304)              0         
                                                                 
 dense_3 (Dense)             (None, 64)               

In [ ]:
print("Dokładność predykcji na zbiorze walidacyjnym wynosi :", history.history['val_accuracy'])
print("Dokładność predykcji na zbiorze treningowym wynosi:", history.history['accuracy'])
print("Strata na zbiorze walidacyjnym :", history.history['val_loss'])
print("Strata na zbiorze treningowym :", history.history['loss'])

Dokładność predykcji na zbiorze walidacyjnym wynosi : [0.9848436117172241, 0.986374020576477, 0.9859444499015808, 0.9841321110725403, 0.9848570227622986]
Dokładność predykcji na zbiorze treningowym wynosi: [0.9866156578063965, 0.9872230887413025, 0.9877634644508362, 0.9880118370056152, 0.9879010319709778]
Strata na zbiorze walidacyjnym : [0.06463632732629776, 0.06621440500020981, 0.0642656683921814, 0.07897284626960754, 0.07789570838212967]
Strata na zbiorze treningowym : [0.05313950777053833, 0.05218150094151497, 0.05057717114686966, 0.0486755296587944, 0.05138608068227768]
